# 1. 데이터 불러오기

In [1]:
# 필요한 모듈 불러오기
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메세지 무시

In [2]:
# csv 파일을 하나씩 데이터 프레임으로 변환
df1 = pd.read_csv('D:/박상춘/프로젝트/raw data/월별 행정동별 소비 데이터/delivery_서울특별시_202209.csv', encoding='CP949', low_memory=False)
df2 = pd.read_csv('D:/박상춘/프로젝트/raw data/월별 행정동별 소비 데이터/delivery_서울특별시_202210.csv', encoding='CP949', low_memory=False)
df3 = pd.read_csv('D:/박상춘/프로젝트/raw data/월별 행정동별 소비 데이터/delivery_서울특별시_202211.csv', encoding='CP949', low_memory=False)

In [3]:
# 데이터 프레임을 하나로 합치기
df = pd.concat([df1, df2, df3])

# 2. 데이터 확인 및 전처리

In [4]:
df.head()

,crym,admd_c,ana_mgpo_nm,ana_ccd_nm,admd_nm,dayofweek,time_interval,sl_am,sl_ct
0,202209,1111051500,서울특별시,종로구,청운효자동,Fri,0,45700.0,2
1,202209,1111051500,서울특별시,종로구,청운효자동,Fri,8,16900.0,1
2,202209,1111051500,서울특별시,종로구,청운효자동,Fri,9,35400.0,1
3,202209,1111051500,서울특별시,종로구,청운효자동,Fri,10,73100.0,4
4,202209,1111051500,서울특별시,종로구,청운효자동,Fri,11,92400.0,4


In [5]:
# 필요없는 컬럼 제거
df.drop(['admd_c', 'ana_mgpo_nm'], axis=1, inplace=True)

In [6]:
# 컬럼명 바꾸기
df.rename({'crym':'date', 'ana_ccd_nm':'gu', 'admd_nm':'dong', 'dayofweek':'days', 'time_interval':'time_int', 'sl_am':'order_amt', 'sl_ct':'order_cnt'}, axis=1, inplace=True)

In [7]:
df.drop('dong', axis=1, inplace=True)

In [8]:
df.head()

,date,gu,days,time_int,order_amt,order_cnt
0,202209,종로구,Fri,0,45700.0,2
1,202209,종로구,Fri,8,16900.0,1
2,202209,종로구,Fri,9,35400.0,1
3,202209,종로구,Fri,10,73100.0,4
4,202209,종로구,Fri,11,92400.0,4


In [9]:
# 날짜 및 자치구 리스트
date_arr = df.date.unique()
date_lis = date_arr.tolist()
gu_arr = df.gu.unique()
gu_lis = gu_arr.tolist()

In [15]:
lis = []
for date in date_lis:
    df_t = df[df['date'] == date] # 월별 임시 데이터 프레임에 저장
    for gu in gu_lis:
        df_t1 = df_t[df_t['gu'] == gu] # 구별 임시 데이터 프레임에 저장

        for i in range(24):
            df_mon = df_t1[(df_t1['days'] == 'Mon') & (df_t1['time_int'] == i)]
            df_tue = df_t1[(df_t1['days'] == 'Tue') & (df_t1['time_int'] == i)]
            df_wed = df_t1[(df_t1['days'] == 'Wed') & (df_t1['time_int'] == i)]
            df_thu = df_t1[(df_t1['days'] == 'Thu') & (df_t1['time_int'] == i)]
            df_fri = df_t1[(df_t1['days'] == 'Fri') & (df_t1['time_int'] == i)]
            df_sat = df_t1[(df_t1['days'] == 'Sat') & (df_t1['time_int'] == i)]
            df_sun = df_t1[(df_t1['days'] == 'Sun') & (df_t1['time_int'] == i)]

            lis.append([date, gu, '월요일', i, df_mon['order_amt'].sum(), df_mon['order_cnt'].sum()])
            lis.append([date, gu, '화요일', i, df_tue['order_amt'].sum(), df_tue['order_cnt'].sum()])
            lis.append([date, gu, '수요일', i, df_wed['order_amt'].sum(), df_wed['order_cnt'].sum()])
            lis.append([date, gu, '목요일', i, df_thu['order_amt'].sum(), df_thu['order_cnt'].sum()])
            lis.append([date, gu, '금요일', i, df_fri['order_amt'].sum(), df_fri['order_cnt'].sum()])
            lis.append([date, gu, '토요일', i, df_sat['order_amt'].sum(), df_sat['order_cnt'].sum()])
            lis.append([date, gu, '일요일', i, df_sun['order_amt'].sum(), df_sun['order_cnt'].sum()])

In [16]:
df1 = pd.DataFrame(lis, columns=['date', 'gu', 'days', 'time_int', 'order_amt', 'order_cnt'])

In [17]:
df1.head()

,date,gu,days,time_int,order_amt,order_cnt
0,202209,종로구,월요일,0,224080.0,8
1,202209,종로구,화요일,0,81270.0,4
2,202209,종로구,수요일,0,312520.0,12
3,202209,종로구,목요일,0,104100.0,4
4,202209,종로구,금요일,0,314773.0,14


In [18]:
# 데이터 타입 확인
df1.dtypes

date           int64
gu            object
days          object
time_int       int64
order_amt    float64
order_cnt      int64
dtype: object

In [19]:
# 결측치 확인
df1.isnull().sum()

date         0
gu           0
days         0
time_int     0
order_amt    0
order_cnt    0
dtype: int64

In [20]:
# 인덱스 리셋
df1.set_index([pd.Index(range(0,df1.gu.count()))], inplace=True)

In [21]:
# 메모리 용량 확인
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       12600 non-null  int64  
 1   gu         12600 non-null  object 
 2   days       12600 non-null  object 
 3   time_int   12600 non-null  int64  
 4   order_amt  12600 non-null  float64
 5   order_cnt  12600 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 590.8+ KB


# 3. csv 파일로 변환

In [22]:
df1.to_csv('./csv/monthly_dong_order.csv')